<a href="https://colab.research.google.com/github/JozefSL/EIA/blob/main/Power/Daily/L48/Demand930.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
#import os

In [105]:
from google.colab import userdata
api_key = userdata.get('api_key')

In [106]:
url = 'https://api.eia.gov/v2/electricity/rto/daily-region-data/data/?api_key='+ api_key + '&data[0]=value&sort[0][column]=period&sort[0][direction]=asc'

In [114]:
f1 = '&start=2019-01-01'
f2 = '&frequency=daily'
f3 = '&facets[respondent][]=US48'
f4 = '&facets[timezone][]=Pacific'
f5 = '&facets[type][]=NG'

In [115]:
response = requests.get(url + f1 + f2 + f3 +f4 + f5)

In [161]:
df = pd.DataFrame(response.json().get('response').get('data'))[['period','respondent','timezone','type','value']]
df.columns = ['period','respondent','timezone','type','value']
df['period'] = df['period'].apply(pd.to_datetime)
df.value = (df.value.astype('Int64')/1000000).round(1) #.astype('Int64')
df['year'] = df['period'].dt.year
df['week'] = df['period'].dt.isocalendar().week
df['change'] = df['value'] - df['value'].shift(1)
ty = df.year.iloc[-1]
ly = ty-1
df.tail(3)

,period,respondent,timezone,type,value,year,week,change
2107,2024-10-08,US48,Pacific,NG,10.4,2024,41,-0.4
2108,2024-10-09,US48,Pacific,NG,10.4,2024,41,0.0
2109,2024-10-10,US48,Pacific,NG,1.1,2024,41,-9.3


In [131]:
#df[(df['type']=='NG') & (df['timezone']=='Pacific')]

In [162]:
#dfA = df[df['type']=='NG'].pivot(index='period', columns='timezone', values='value').round(1) #.astype('Int64') #.reset_index(drop='true')
#dfA = df.pivot(index='week', columns='year', values='value').round(1)
#dfA = dfA.drop(dfA.index[-1]).reset_index()  #eliminate the add years with 53 weeks
dfA = df.groupby(['week','year']).agg({'value':'sum'}).sort_values(by=['year','week']).reset_index()
dfA = dfA.drop(dfA.index[-1])
dfA = dfA.pivot(index='week', columns='year', values='value')
dfA = dfA.drop(dfA.index[-1]).reset_index()  #eliminate the add years with 53 weeks
dfA.tail(3)

year,week,2019,2020,2021,2022,2023,2024
49,50,74.4,72.9,73.8,78.8,77.8,<NA>
50,51,77.5,78.1,73.1,89.4,76.7,<NA>
51,52,66.6,74.1,51.4,86.2,82.9,<NA>


In [163]:
dfAA = pd.DataFrame(dfA.iloc[:, -6:-1].mean(axis=1))
dfAA.columns = ['5yAvg']
dfAA['5yMin'] = dfA.iloc[:, -6:-1].min(axis=1)
dfAA['5yMaxA'] = dfA.iloc[:, -6:-1].max(axis=1)
dfAA['5yMax'] = dfAA['5yMaxA']-dfAA['5yMin']
dfAA['week'] = dfA['week']

In [164]:
dfA['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfA['week'].astype(str)+'5',format='%Y%W%w')
dfAA['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAA['week'].astype(str)+'5',format='%Y%W%w')

In [165]:
dfAA.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,week,weekDate
0,72.06,49.0,81.1,32.1,1,2024-01-05
1,76.06,71.9,80.3,8.4,2,2024-01-12
2,76.74,73.1,83.9,10.8,3,2024-01-19


In [168]:
# Create the area chart
fig = px.area(dfAA, x=dfAA.weekDate, y=['5yMin', '5yMax'], range_y=[60, 100], #range_x=[dfAA.index.min()-.2, dfAA.index.max()+.2],
              title='Stacked Area Chart with Line Chart').update_traces(fillcolor='rgba(128, 128, 128, 0.2)')

fig.update_traces(line_color='rgba(0,0,0,0)',showlegend=False)

fig.add_scatter(x=dfAA.weekDate, y=dfAA['5yAvg'], mode='lines', name='5yAvg', line=dict(color='darkgrey', width=2))
fig.add_scatter(x=dfA.weekDate, y=dfA[ly], mode='lines', name=str(ly),line=dict(color='blue', width=2))
fig.add_scatter(x=dfA.weekDate, y=dfA[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

# Customize the chart
fig.update_layout(
    autosize=False,
    width=1200,
    height=800,
    title={
        'text': '<b>Pacific Net Generation Weekly Sum</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    xaxis_title='Week',
    yaxis_title='Pacific Net Generation Weekly (PWhr)'

)


fig.update_traces(fillcolor='rgba(0,0,0,0)', selector=dict(name='5yMin'))


fig.add_scatter(x = [dfA['weekDate'][dfA[ty].isna().idxmax()-1]], y = [dfA[ty][dfA[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = [dfA[ty][dfA[ty].isna().idxmax()-1]],
                     textfont=dict(color='red', size=16),
                     textposition='bottom right') # top, bottom, middle


fig.show()

In [178]:
dfAc = df.groupby(['week','year']).agg({'change':'sum'}).sort_values(by=['year','week']).reset_index()
dfAc = dfAc.drop(dfAc.index[-1])
dfAc = dfAc.pivot(index='week', columns='year', values='change')
dfAc = dfAc.drop(dfAc.index[-1]).reset_index()  #eliminate the add years with 53 weeks
dfAc.tail(3)

year,week,2019,2020,2021,2022,2023,2024
49,50,0.3,-0.2,0.2,1.3,-0.1,<NA>
50,51,0.1,0.4,-1.1,1.1,-0.6,<NA>
51,52,-0.9,0.1,0.5,-2.2,0.5,<NA>


In [180]:
dfAAc = pd.DataFrame(dfAc.iloc[:, -6:-1].mean(axis=1))
dfAAc.columns = ['5yAvg']
dfAAc['5yMin'] = dfAc.iloc[:, -6:-1].min(axis=1) # Actual minimum, also used to calculate stack area bottom
dfAAc['5yMaxA'] = dfAc.iloc[:, -6:-1].max(axis=1) # Actual maximum, also used to calculate stack area top
dfAAc['5yMax'] = dfAAc['5yMaxA']-dfAAc['5yMin']
dfAAc['week'] = dfAc['week']
dfAAc.head(3)

,5yAvg,5yMin,5yMaxA,5yMax,week
0,0.56,-0.1,1.0,1.1,1
1,0.28,-0.7,0.9,1.6,2
2,0.26,-0.2,0.8,1.0,3


In [181]:
dfAc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAc['week'].astype(str)+'5',format='%Y%W%w')
dfAAc['weekDate'] = pd.to_datetime(str(pd.Timestamp.now().year)+dfAAc['week'].astype(str)+'5',format='%Y%W%w')

In [182]:
categories = dfAAc['weekDate']
lower_bounds = dfAAc['5yMin']
upper_bounds = dfAAc['5yMaxA']
last_valueCH = [dfAc[ty][dfAc[ty].isna().idxmax()-1]]

In [200]:
fig = go.Figure()

for i in range(len(categories)):
    fig.add_trace(go.Bar(
        x=[categories[i]],
        y=[upper_bounds[i] - lower_bounds[i]],
        base=[lower_bounds[i]],
        marker=dict(color='lightgrey'),
        showlegend=False
        #color_discrete_sequence=['blue']
        #name=str(categories[i])
    ))

fig.add_scatter(x=dfAAc.weekDate, y=dfAAc['5yAvg'], mode='markers', name='5yAvg', line=dict(color='grey', width=2))
fig.add_scatter(x=dfAc.weekDate, y=dfAc[ly], mode='markers', name=str(ly),line=dict(color='blue', width=2))
fig.add_scatter(x=dfAc.weekDate, y=dfAc[ty], mode='lines+markers', name=str(ty),line=dict(color='red', width=3), marker=dict(color='red'))

fig.add_scatter(x = [dfAc['weekDate'][dfAc[ty].isna().idxmax()-1]], y = [dfAc[ty][dfAc[ty].isna().idxmax()-1]],
                     mode = 'markers + text',
                     marker = {'color':'red', 'size':14},
                     showlegend = False,
                     text = f"{dfAc[ty][dfAc[ty].isna().idxmax()-1]:.3f}",
                     textfont=dict(color='red', size=16),
                     textposition='top center') # top left, bottom right, middle left, bottom righ, top center, auto center

fig.update_layout(autosize=False,width=1200, height=700,
    title={
        'text': '<b>Pacific Net Generation Weekly Changes</b>',
        'font': {'size': 28, 'family': 'Arial', 'color': 'black'}
        },
    #xaxis_title='Week',
    yaxis_title='Pacific Net Generation Weekly (PWhr)')

# Add source link as an annotation
fig.add_annotation(
    text="Source: EIA 930",
    xref="paper", yref="paper",
    x=0.95, y=0.05,
    showarrow=False,
    font=dict(size=13)
)

fig.show()